In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Los datos corresponden a un subset del dataset cats vs dogs de https://www.kaggle.com/c/dogs-vs-cats
files=glob.glob('/content/drive/MyDrive/Machine_Learning/CATS_DOGS/*.jpg')

In [ ]:
size = 64

train_data = np.array([
    np.asarray(Image.open(file).resize((size, size)))
    for file in files[:200]
])

train_label = np.array([
    1 if 'dog' in file else 0
    for file in files[:200]
])

test_label = np.array([
    1 if 'dog' in file else 0
    for file in files[200:]
])

test_data = np.array([
    np.asarray(Image.open(file).resize((size, size)))
    for file in files[200:]
])

In [ ]:
train_data_flatten = train_data.reshape(train_data.shape[0], -1).T
train_set_label=train_label.reshape((1,train_label.shape[0]))
#test_set_label=test_label.reshape((1,test_label.shape[0])) #Use test_label
test_data_flatten = test_data.reshape(test_data.shape[0], -1).T

In [ ]:
#Normalization
train_set_data= train_data_flatten/255
test_set_data = test_data_flatten/np.max(test_data_flatten)

In [ ]:
def sigmoid(z):
  a = 1/(1+np.exp(-z))
  return a

def tanh(z):
  a = np.tanh(z)
  return a

def tanh_grad(x, y, iteraciones, alpha):
  w = np.random.randn(x.shape[0], 1) * np.sqrt(1 / x.shape[0])
  b = np.random.randn()
  m = x.shape[1]
  for i in range(iteraciones):
    suma = 0
    a = tanh(np.matmul(w.T,x)+b)
    dz = (a - y) * (1 - a**2)
    dw = (1/m) * np.dot(x, dz.T)
    db = (1/m) * np.sum(dz)
    w = w - alpha*dw
    b = b - alpha*db
    epsilon = 1e-8
    a_clipped = np.clip(a, epsilon, 1 - epsilon)
    J = -(1/m) * np.sum(y * np.log(a_clipped) + (1 - y) * np.log(1 - a_clipped))
    f = i%(iteraciones/10)
    if f==0:
      print('J en la ',i,'ésima iteración: ',J)
  return w, b, J

def sigmoid_grad(x,y,w, b, iteraciones, alpha):
  m = x.shape[1]
  #w,b = np.zeros((len(train_set_data),1)),0 #Inicializamos los pesos en cero
  for i in range(iteraciones):
    suma = 0 #Defino la variable donde guardaré el valor de la sumatoria para el cálculo del costo
    a = sigmoid(np.matmul(w,x)+b.reshape(-1, 1)) #Calcula la función de activación con w,b iguales a cero en principio
    dw = (1/m) * np.matmul((a - y), x.T)  # dw: (3, 12288)
    w = w - alpha * dw
    db = np.mean(a - y)
    b = b - alpha * db #Actualizamos b usando db
    #b = b-alpha*(1/m)*(a-y) #Actualizamos b usando db
    for j in range(a.shape[1]): #Hacemos el cálculo del costo con el w,b iniciales
      suma += y[:1,j]*np.log(a[:1,j])+(1-y[:1,j])*np.log(1-(a[:1,j]))
    J = -(1/m)*sum(suma) #Calculamos el costo
    f = i%(iteraciones/10)
    if f == 0:
      print('J en la ',i,'-ésima iteracion:', J) #Imprimimos el costo en cada iteración
  return w, b, J

def binary(x, y, w, b):
  ytst = sigmoid(np.matmul(w, x)+b.reshape(-1,1))
  return ytst

def precission(data, labels):
  y_test = np.zeros_like(data)
  for j in range(3):
    for i in range(len(data[0].shape)):
      if data[j, i] > 0.5:
        y_test[j,i]=1
      else:
        y_test[j,i]=0

  from sklearn.metrics import accuracy_score
  y_pred = np.argmax(y_test, axis=0)
  accuracy = accuracy_score(y_pred, labels)
  print('La precisión es del ',accuracy,'%')

  return accuracy

def NN(x_train, y_train, x_test, y_test, iteraciones, alpha):
    w_list = []
    b_list = []
    for i in range(3):  # Número de neuronas
        omega, bias, J = tanh_grad(x_train, y_train, iteraciones, alpha)
        w_list.append(omega)
        b_list.append(bias)

    w_array = np.array(w_list)  # Forma: (3, 12288, 1)
    b_array = np.array(b_list)  # Forma: (3,)

    w, b, J = sigmoid_grad(x_train, y_train,w_array[:,:,0], b_array, iteraciones, alpha)

    precision_test = binary(x_test, y_test, w, b)

    accuracy = precission(precision_test, y_test)

    return w, b

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def tanh(z):
    return np.tanh(z)

def initialize_parameters(n_x, n_h):
    W1 = np.random.randn(n_h, n_x) * np.sqrt(1. / n_x)
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(1, n_h) * np.sqrt(1. / n_h)
    b2 = np.zeros((1, 1))
    return W1, b1, W2, b2

def forward_propagation(X, W1, b1, W2, b2):
    Z1 = np.dot(W1, X) + b1
    A1 = tanh(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    return A1, A2

def compute_cost(A2, Y):
    m = Y.shape[1]
    epsilon = 1e-8
    A2 = np.clip(A2, epsilon, 1 - epsilon)
    return -1/m * np.sum(Y * np.log(A2) + (1 - Y) * np.log(1 - A2))

def backward_propagation(X, Y, A1, A2, W2):
    m = X.shape[1]
    dZ2 = A2 - Y
    dW2 = (1/m) * np.dot(dZ2, A1.T)
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)

    dA1 = np.dot(W2.T, dZ2)
    dZ1 = dA1 * (1 - A1**2)
    dW1 = (1/m) * np.dot(dZ1, X.T)
    db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)

    return dW1, db1, dW2, db2

def update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 -= alpha * dW1
    b1 -= alpha * db1
    W2 -= alpha * dW2
    b2 -= alpha * db2
    return W1, b1, W2, b2

def predict(A2):
    return (A2 > 0.5).astype(int)

def accuracy(Y_pred, Y_true):
    return accuracy_score(Y_true.flatten(), Y_pred.flatten())

def neural_network(X, Y, X_test, Y_test, n_h, iterations, alpha):
    n_x = X.shape[0]
    W1, b1, W2, b2 = initialize_parameters(n_x, n_h)

    for i in range(iterations):
        A1, A2 = forward_propagation(X, W1, b1, W2, b2)
        cost = compute_cost(A2, Y)
        dW1, db1, dW2, db2 = backward_propagation(X, Y, A1, A2, W2)
        W1, b1, W2, b2 = update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)

        if i % (iterations // 10) == 0:
            print(f'Costo en la iteración {i}: {cost:.4f}')

    _, A2_test = forward_propagation(X_test, W1, b1, W2, b2)
    Y_pred = predict(A2_test)
    acc = accuracy(Y_pred, Y_test)

    print(f'Precisión final en test: {acc:.4f}')
    return W1, b1, W2, b2, acc


In [ ]:
W1, b1, W2, b2, acc = neural_network(train_set_data, train_set_label,test_set_data,test_label, n_h=3, iterations=10000, alpha=0.01)

Costo en la iteración 0: 0.7041
Costo en la iteración 1000: 0.1919
Costo en la iteración 2000: 0.0577
Costo en la iteración 3000: 0.0302
Costo en la iteración 4000: 0.0205
Costo en la iteración 5000: 0.0155
Costo en la iteración 6000: 0.0124
Costo en la iteración 7000: 0.0103
Costo en la iteración 8000: 0.0089
Costo en la iteración 9000: 0.0078
Precisión final en test: 0.5213


In [ ]:
w, b = NN(train_set_data, train_set_label,test_set_data,test_label,  1000, 0.01)

J en la  0 ésima iteración:  11.236615257710946
J en la  100 ésima iteración:  7.043000882559377
J en la  200 ésima iteración:  7.04287202396657
J en la  300 ésima iteración:  7.04274282907563
J en la  400 ésima iteración:  7.042613296098996
J en la  500 ésima iteración:  7.042483423301193
J en la  600 ésima iteración:  7.0423532088833385
J en la  700 ésima iteración:  7.042222651016021
J en la  800 ésima iteración:  7.042091747880553
J en la  900 ésima iteración:  7.041960497694228
J en la  0 ésima iteración:  0.7531965910655696
J en la  100 ésima iteración:  7.161712231600566
J en la  200 ésima iteración:  7.161712179788638
J en la  300 ésima iteración:  7.161712127975977
J en la  400 ésima iteración:  7.161712076163219
J en la  500 ésima iteración:  7.161712024349726
J en la  600 ésima iteración:  7.1617119725355005
J en la  700 ésima iteración:  7.161711920720543
J en la  800 ésima iteración:  7.1617118689054875
J en la  900 ésima iteración:  7.161711817089697
J en la  0 ésima iter